<div style="border-left: 6px solid #7B61FF; color:white; padding:20px; border-radius:10px; font-family:Arial, sans-serif; text-align:center; font-size:28px; font-weight:bold;">
  🧱 02 – Baseline Model
</div>